# Importing libraries

In [ ]:
!pip install -qqq bitsandbytes torch transformers peft accelerate datasets loralib einops trl

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.1/76.1 MB 11.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 111.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 68.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 37.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 13.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 42.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 485.4/485.4 kB 33.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
!pip install evaluate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 8.0 MB/s eta 0:00:00


In [ ]:
!pip install rouge_score
!pip install bert_score

  Preparing metadata (setup.py) ... done
  Created wheel for rouge_score: filename=rouge_score-0.1.2-py3-none-any.whl size=24935 sha256=ed1246a3af26c5b524292fa3e9ae79c8b2def28fb3de20c163d1111ff57bf4c9
  Stored in directory: /root/.cache/pip/wheels/1e/19/43/8a442dc83660ca25e163e1bd1f89919284ab0d0c1475475148
Successfully built rouge_score
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.1/61.1 kB 5.2 MB/s eta 0:00:00


In [ ]:
# Install Kaggle API
!pip install kaggle

In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
from datasets import load_dataset
from peft import LoraConfig, get_peft_model
import torch
import evaluate
from transformers import Trainer, TrainingArguments
from sklearn.model_selection import train_test_split
import json
import glob
import os
import pandas as pd
from transformers import TrainerCallback
import bitsandbytes as bnb


# **preparing llm**

In [ ]:
!pip install colab-xterm
%load_ext colabxterm

Type these codes in below terminal after run the cell (%xterm)
* curl -fsSL https://ollama.com/install.sh | sh
* ollama serve & ollama pull llama3.2

In [ ]:
%xterm

Launching Xterm...

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


🚀  Listen to 10000
{"success": true, "reason": null}



In [ ]:
!pip -qq install langchain
!pip -qq install langchain-core
!pip -qq install langchain-community

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [ ]:
from langchain.llms import Ollama
llm = Ollama(model = "llama3.2")

/tmp/ipykernel_460626/830612665.py:2: LangChainDeprecationWarning: The class `Ollama` was deprecated in LangChain 0.3.1 and will be removed in 1.0.0. An updated version of the class exists in the :class:`~langchain-ollama package and should be used instead. To use it run `pip install -U :class:`~langchain-ollama` and import as `from :class:`~langchain_ollama import OllamaLLM``.
  llm = Ollama(model = "llama3.2")


try if it works for german:

In [ ]:
out = llm.invoke('Was sind Ferien?')
print(out)

# Dataset

upload here **'kaggle.json'** file if needed when running the first cell 

In [ ]:
# from google.colab import files
# files.upload()

# Move kaggle.json to the proper directory
!mkdir -p ~/.kaggle
!mv kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json  # Secure the file

# Download the dataset
!kaggle datasets download -d mexwell/german-political-speeches-corpus

# Unzip the dataset
!unzip german-political-speeches-corpus.zip -d german_speeches

# Verify the file is present
os.listdir("german_speeches")


mv: cannot stat 'kaggle.json': No such file or directory
chmod: cannot access '/root/.kaggle/kaggle.json': No such file or directory
Dataset URL: https://www.kaggle.com/datasets/mexwell/german-political-speeches-corpus
License(s): Attribution 4.0 International (CC BY 4.0)
 99% 28.0M/28.3M [00:02<00:00, 15.9MB/s]
100% 28.3M/28.3M [00:02<00:00, 10.5MB/s]
Archive:  german-political-speeches-corpus.zip
  inflating: german_speeches/AuswartigesAmt.xml  
  inflating: german_speeches/Bundesprasidenten.xml  
  inflating: german_speeches/Bundesregierung.xml  
  inflating: german_speeches/Bundestagsprasidenten.xml  


['Bundesprasidenten.xml',
 'Bundestagsprasidenten.xml',
 'AuswartigesAmt.xml',
 'Bundesregierung.xml']

In [ ]:
df1 = pd.read_xml('german_speeches/AuswartigesAmt.xml')
df2 = pd.read_xml('german_speeches/Bundesprasidenten.xml')
df3 = pd.read_xml('german_speeches/Bundesregierung.xml')
df4 = pd.read_xml('german_speeches/Bundestagsprasidenten.xml')

In [ ]:
dataframes = [df1, df2, df3, df4]
df = pd.concat(dataframes, ignore_index=True).iloc[:5000] #concatentante all dataframes into 1 and then take the first 5000 doc only

In [ ]:
df

,person,titel,datum,ort,untertitel,url,anrede,rohtext
0,Cornelia Pieper,"Rede Staatsministerin Pieper: ""150 Jahre Wisse...",2010-10-14,NaN,"""150 Jahre Wissenschaftsbeziehungen Deutschlan...",http://www.auswaertiges-amt.de/DE/Infoservice/...,"Sehr geehrter Herr Präsident Kajiyama,meine Da...",\nIch freue mich sehr über den warmherzigen Em...
1,Cornelia Pieper,Rede von Staatsministerin Cornelia Pieper im D...,2011-06-30,NaN,Rede von Staatsministerin Cornelia Pieper im D...,http://www.auswaertiges-amt.de/DE/Infoservice/...,Liebe Kolleginnen und Kollegen!,"\nIch begrüße es, dass wir heute über die weit..."
2,Guido Westerwelle,Rede von Guido Westerwelle im Deutschen Bundes...,2010-05-21,NaN,Rede von Bundesaußenminister Guido Westerwelle...,http://www.auswaertiges-amt.de/DE/Infoservice/...,NaN,"\nFrau Präsidentin!\nHerr Kollege Gabriel, Ihr..."
3,Frank-Walter Steinmeier,Rede von Außenminister Steinmeier anlässlich d...,2014-11-15,NaN,Rede von Außenminister Steinmeier anlässlich d...,http://www.auswaertiges-amt.de/DE/Infoservice/...,"Sehr geehrte Damen und Herren,",\nich sage Dank für die Einladung und habe Res...
4,Guido Westerwelle,Rede von Außenminister Westerwelle vor der Ver...,2010-11-02,NaN,Rede von Außenminister Westerwelle vor der Ver...,http://www.auswaertiges-amt.de/DE/Infoservice/...,"Sehr geehrte Damen und Herren,",\nzunächst möchte ich mich sehr herzlich bei I...
...,...,...,...,...,...,...,...,...
4995,Angela Merkel,Rede von Bundeskanzlerin Angela Merkel anlässl...,2011-02-01,None,in Tel Aviv,http://www.bundesregierung.de/nn_1498/Content/...,NaN,"\nSehr geehrter Herr Präsident,\nsehr geehrte ..."
4996,Angela Merkel,Rede von Bundeskanzlerin Angela Merkel anlässl...,2011-02-01,None,in Tel Aviv,http://www.bundesregierung.de/nn_1498/Content/...,NaN,"\nSehr geehrter Herr Vorsitzender,\nsehr geehr..."
4997,Angela Merkel,Rede der Bundeskanzlerin auf der 47. Münchner ...,2011-02-05,None,in München,http://www.bundesregierung.de/nn_1498/Content/...,"Meine Damen und Herren,","\nlieber Herr Ischinger,\nlieber David Cameron..."
4998,Angela Merkel,Rede von Bundeskanzlerin Angela Merkel anlässl...,2011-02-08,None,in Berlin,http://www.bundesregierung.de/nn_1498/Content/...,"Meine Damen und Herren,","\nliebe Kristina Schröder,\nliebe Gäste der he..."


In [ ]:
df=  df.iloc[:, -1:] #keep only the column of the texts to be summarized
print(df)

                                                rohtext
0     \nIch freue mich sehr über den warmherzigen Em...
1     \nIch begrüße es, dass wir heute über die weit...
2     \nFrau Präsidentin!\nHerr Kollege Gabriel, Ihr...
3     \nich sage Dank für die Einladung und habe Res...
4     \nzunächst möchte ich mich sehr herzlich bei I...
...                                                 ...
4995  \nSehr geehrter Herr Präsident,\nsehr geehrte ...
4996  \nSehr geehrter Herr Vorsitzender,\nsehr geehr...
4997  \nlieber Herr Ischinger,\nlieber David Cameron...
4998  \nliebe Kristina Schröder,\nliebe Gäste der he...
4999  \nich glaube, ich darf sagen, dass hier in die...

[5000 rows x 1 columns]


trying on one of the documents:

In [ ]:
prompt = f'''Fasse den folgenden Text zusammen {df['rohtext'][0]}'''
out = llm.invoke(prompt)
print(out)

Der Text ist eine Rede des deutschen Staatsministers im Auswärtigen Amt an die Dokkyo-Universität in Japan. Sie soll eine Ehrung für die Universität erhalten, da sie sich um die Förderung der deutsch-japanischen Beziehungen bemüht.

Die Rede beginnt mit einer Betonung des historischen Kontexts der deutsch-japanischen Beziehungen und der Bedeutung dieser Beziehungen für die Zukunft. Sie betont die gemeinsamen Herausforderungen, denen sich die beiden Länder gegenübersehen müssen.

Der Sprecher erinnert sich an die Philipp Franz von Siebold-Preise, die jährlich an junge japanische Forscher verliehen werden, um ihre wissenschaftlichen Leistungen zu belohnen. Diese Preise sollen die Stärkung der deutsch-japanischen Beziehungen fördern.

Die Rede enthält auch eine Bitte an die Studierenden der Universität, sich um die Förderung der deutschen Sprache und Kultur in Japan zu bemühen, da sie die Zukunft der Beziehungen prägen werden. Der Sprecher wünscht den Studenten viel Erfolg bei ihren Ausla

# Baseline summaries:

no need to run again

Just upload the **'all_summaries.json'** file which contains the 5000 baseline summaries already generated by the LLM llama3.2

In [ ]:
# Split the dataframe into smaller batches
batch_size = 50
texts = df['rohtext'].tolist()  # Convert the column to a list

#summarize a batch of texts
def summarize_batch(batch):
    prompts = [f'''Fasse den folgenden Text zusammen {text}''' for text in batch]
    outputs = [llm.invoke(prompt) for prompt in prompts]
    return outputs

# Process the texts in batches
nb = 1
for i in range(0, len(texts), batch_size):
    batch = texts[i:i + batch_size]
    print(f'Processing batch {nb} with {len(batch)} documents...')

    # Get the summaries for the batch
    batch_summaries = summarize_batch(batch)

    # Save the summaries for this batch to a JSON file (one summary per line)
    file_name = f'summaries_batch_{nb}.json'
    with open(file_name, 'w', encoding='utf-8') as file:
        for summary in batch_summaries:
            if summary:  # Check if summary is not empty
                # Save each summary on a new line in the JSON file
                json.dump({"summary": summary}, file, ensure_ascii=False)
                file.write('\n')  # New line after each summary

    print(f'Batch {nb} completed and saved to {file_name}.')
    nb += 1

print(f'Summarization completed. All batches saved separately.')


concatenate:

In [ ]:
# Define the pattern to match all summary files
file_pattern = "summaries_batch_*.json"

# Get a sorted list of all summary files
summary_files = sorted(glob.glob(file_pattern), key=lambda x: int(x.split('_')[-1].split('.')[0]))

# List to store all summaries
all_summaries = []

# Read each file and extract summaries
for file_name in summary_files:
    with open(file_name, 'r', encoding='utf-8') as file:
        summaries = [json.loads(line)["summary"] for line in file]  # Extract summary from each line
        all_summaries.extend(summaries)  # Add to the main list

# Save all summaries to a single JSON file
output_file = "all_summaries.json"
with open(output_file, 'w', encoding='utf-8') as file:
    json.dump(all_summaries, file, ensure_ascii=False, indent=4)

print(f"All {len(all_summaries)} summaries merged into {output_file}.")


All 5000 summaries merged into all_summaries.json.


# df with summaries

In [ ]:
#read the json file where we have all 5000 summaries from llm
with open('all_summaries.json', 'r') as f:
    summaries = json.load(f)

#ensure we have summarizations for all doc
if len(summaries) == len(df):
    df['Summarization'] = summaries
else:
    print("The number of summaries doesn't match the number of rows in the DataFrame.")

# Save the updated DataFrame with the new summarization column
df.to_csv('updated_with_summaries.csv', index=False)

<ipython-input-11-94d134926997>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Summarization'] = summaries


In [ ]:
df

,rohtext,Summarization
0,\nIch freue mich sehr über den warmherzigen Em...,Dies ist eine Rede des deutschen Staatsministe...
1,"\nIch begrüße es, dass wir heute über die weit...",Der Bundesminister für Auswärtige Angelegenhei...
2,"\nFrau Präsidentin!\nHerr Kollege Gabriel, Ihr...",Dieser Text handelt von einer Rede des Bundesf...
3,\nich sage Dank für die Einladung und habe Res...,Der Text handelt von der Notwendigkeit einer Q...
4,\nzunächst möchte ich mich sehr herzlich bei I...,Der Text beschreibt eine Rede eines deutschen ...
...,...,...
4995,"\nSehr geehrter Herr Präsident,\nsehr geehrte ...","Dieser Text ist ein Redetext, in dem Bundeskan..."
4996,"\nSehr geehrter Herr Vorsitzender,\nsehr geehr...",Diese Rede von Bundeskanzlerin Angela Merkel b...
4997,"\nlieber Herr Ischinger,\nlieber David Cameron...",Dieser Text ist ein Redebeitrag von Bundeskanz...
4998,"\nliebe Kristina Schröder,\nliebe Gäste der he...",Es scheint sich hier um eine Rede eines Politi...


# train, validation, test

In [ ]:
# Split into training and remaining (validation + test)
train_df, temp_df = train_test_split(df, test_size=0.3, random_state=42) #70% for training, 30% the rest

# split the remaining into validation and test sets
valid_df, test_df = train_test_split(temp_df, test_size=0.5, random_state=42) # 15% for validation & 15% for testing

# Save the splits to CSV files
train_df.to_csv('train_data.csv', index=False)
valid_df.to_csv('valid_data.csv', index=False)
test_df.to_csv('test_data.csv', index=False)


# SLM finetuning


In [ ]:
import bitsandbytes as bnb

In [ ]:
# Loading the model and tokenizer for Llama-3.2-1B
from transformers import AutoModelForSeq2SeqLM

# model_name = "unsloth/Llama-3.2-1B"

model_name= 'sshleifer/distilbart-cnn-12-6'

# Set up the BitsAndBytes configuration for 4-bit quantization
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True, # Use 4-bit precision to load the model, which reduces memory usage
    bnb_4bit_compute_dtype="bfloat16", # Use bfloat16 for computation
    bnb_4bit_quant_type="nf4",  # Use the 'nf4' quantization type for better efficiency
    bnb_4bit_use_double_quant=True # Enable double quantization for further memory reduction
)

# Load the pre-trained model with the specified configuration
model = AutoModelForSeq2SeqLM.from_pretrained(model_name, config= bnb_config)

# Load the tokenizer associated with the model
tokenizer = AutoTokenizer.from_pretrained(model_name)

In [ ]:
print(tokenizer.pad_token)

<pad>


In [ ]:
print(model)

BartForConditionalGeneration(
  (model): BartModel(
    (shared): BartScaledWordEmbedding(50264, 1024, padding_idx=1)
    (encoder): BartEncoder(
      (embed_tokens): BartScaledWordEmbedding(50264, 1024, padding_idx=1)
      (embed_positions): BartLearnedPositionalEmbedding(1026, 1024)
      (layers): ModuleList(
        (0-11): 12 x BartEncoderLayer(
          (self_attn): BartSdpaAttention(
            (k_proj): Linear(in_features=1024, out_features=1024, bias=True)
            (v_proj): Linear(in_features=1024, out_features=1024, bias=True)
            (q_proj): Linear(in_features=1024, out_features=1024, bias=True)
            (out_proj): Linear(in_features=1024, out_features=1024, bias=True)
          )
          (self_attn_layer_norm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
          (activation_fn): GELUActivation()
          (fc1): Linear(in_features=1024, out_features=4096, bias=True)
          (fc2): Linear(in_features=4096, out_features=1024, bias=True)
    

In [ ]:
train_dataset = load_dataset("csv", data_files="train_data.csv", split="train")
val_dataset = load_dataset("csv", data_files="valid_data.csv", split="train")
test_dataset= load_dataset("csv", data_files="test_data.csv", split="train")

Generating train split: 0 examples [00:00, ? examples/s]

Generating train split: 0 examples [00:00, ? examples/s]

Generating train split: 0 examples [00:00, ? examples/s]

In [ ]:
print(train_dataset.column_names)
print(len(train_dataset))

['rohtext', 'Summarization']
3500


In [ ]:
config = LoraConfig(
    r=8,
    lora_alpha=16,  # 2*r
    lora_dropout=0.05,
    target_modules=["q_proj", "v_proj", "k_proj", "out_proj"],
    bias="none",
    task_type="SEQ_2_SEQ_LM"
)
#define the model using LoRA configs
model = get_peft_model(model, config)

In [ ]:
def preprocess_function(examples):
    inputs = [doc for doc in examples["rohtext"]]
    model_inputs = tokenizer(inputs, truncation=True)

    # Tokenize the targets aka summaries
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(examples["Summarization"], max_length=512, truncation=True)

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

# Apply the mapping to dataset
train_dataset = train_dataset.map(preprocess_function, batched=True)
val_dataset = val_dataset.map(preprocess_function, batched=True)
test_dataset= test_dataset.map(preprocess_function, batched=True)


Map:   0%|          | 0/3500 [00:00<?, ? examples/s]

/usr/local/lib/python3.11/dist-packages/transformers/tokenization_utils_base.py:3961: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


Map:   0%|          | 0/750 [00:00<?, ? examples/s]

Map:   0%|          | 0/750 [00:00<?, ? examples/s]

In [ ]:
rouge = evaluate.load("rouge")
bertscore= evaluate.load("bertscore")

just an example to try:

In [ ]:
predictions = [
    "The cat sat on the mat.",
    "The dog played in the park."
]

references = [
    "A cat is sitting on the mat.",
    "A dog is playing in the park."
]

In [ ]:
rouge_results = rouge.compute(predictions=predictions, references=references)

# Compute BERTScore
bert_results = bertscore.compute(predictions=predictions, references=references, lang="en")

# Print ROUGE results
print("ROUGE Results:")
print("ROUGE-1 F1 score:", rouge_results['rouge1'])
print("ROUGE-2 F1 score:", rouge_results['rouge2'])
print("ROUGE-L F1 score:", rouge_results['rougeL'])

# Print BERTScore results
print("\nBERTScore Results:")
print("Precision:", bert_results['precision'])
print("Recall:", bert_results['recall'])
print("F1 score:", bert_results['f1'])

Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


ROUGE Results:
ROUGE-1 F1 score: 0.6153846153846153
ROUGE-2 F1 score: 0.3636363636363636
ROUGE-L F1 score: 0.6153846153846153

BERTScore Results:
Precision: [0.9762694835662842, 0.9870266914367676]
Recall: [0.9671717882156372, 0.9782010316848755]
F1 score: [0.9716993570327759, 0.9825940728187561]


In [ ]:
from transformers import DataCollatorForSeq2Seq

In [ ]:
training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=6,
    weight_decay=0.01,
    save_total_limit=3,
    logging_steps=20
)

data_collator = DataCollatorForSeq2Seq(
    tokenizer,
    model=model,
    padding=True,  # Enable padding to the model's max length
    label_pad_token_id=tokenizer.pad_token_id, # Set the padding token for labels
)

# Initialize the Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    # compute_metrics=compute_metrics,
    data_collator=data_collator
)

model.config.use_cache = False

# Train the model
trainer.train()


/users/eleves-b/2024/zeinab.ghamlouch/.local/lib/python3.9/site-packages/transformers/training_args.py:1594: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
No label_names provided for model class `PeftModelForSeq2SeqLM`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.


Epoch,Training Loss,Validation Loss
1,3.582400,3.244334
2,3.278200,2.956854
3,3.082700,2.800889
4,2.899200,2.721434
5,2.897400,2.684064
6,2.915600,2.670369


TrainOutput(global_step=2628, training_loss=3.24434041541461, metrics={'train_runtime': 3358.8945, 'train_samples_per_second': 6.252, 'train_steps_per_second': 0.782, 'total_flos': 3.270909100032e+16, 'train_loss': 3.24434041541461, 'epoch': 6.0})

In [ ]:
# Evaluate the model after training
eval_results = trainer.evaluate()

# Print the evaluation results (ROUGE and BERTScore)
print("Evaluation Results:", eval_results)

Evaluation Results: {'eval_loss': 2.6703693866729736, 'eval_runtime': 45.1512, 'eval_samples_per_second': 16.611, 'eval_steps_per_second': 2.082, 'epoch': 6.0}


In [ ]:
model.save_pretrained("saved_model")
tokenizer.save_pretrained("saved_model")


('saved_model/tokenizer_config.json',
 'saved_model/special_tokens_map.json',
 'saved_model/vocab.json',
 'saved_model/merges.txt',
 'saved_model/added_tokens.json',
 'saved_model/tokenizer.json')

In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [ ]:
model_path = "drive/My Drive/saved_model"


In [ ]:
from transformers import AutoModel, AutoTokenizer

# Load model and tokenizer
model = AutoModel.from_pretrained(model_path)
tokenizer = AutoTokenizer.from_pretrained(model_path)


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/1.80k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/1.22G [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.22G [00:00<?, ?B/s]

Loading adapter weights from drive/My Drive/saved_model led to unexpected keys not found in the model: model.decoder.layers.0.encoder_attn.k_proj.lora_A.default.weight, model.decoder.layers.0.encoder_attn.k_proj.lora_B.default.weight, model.decoder.layers.0.encoder_attn.out_proj.lora_A.default.weight, model.decoder.layers.0.encoder_attn.out_proj.lora_B.default.weight, model.decoder.layers.0.encoder_attn.q_proj.lora_A.default.weight, model.decoder.layers.0.encoder_attn.q_proj.lora_B.default.weight, model.decoder.layers.0.encoder_attn.v_proj.lora_A.default.weight, model.decoder.layers.0.encoder_attn.v_proj.lora_B.default.weight, model.decoder.layers.0.self_attn.k_proj.lora_A.default.weight, model.decoder.layers.0.self_attn.k_proj.lora_B.default.weight, model.decoder.layers.0.self_attn.out_proj.lora_A.default.weight, model.decoder.layers.0.self_attn.out_proj.lora_B.default.weight, model.decoder.layers.0.self_attn.q_proj.lora_A.default.weight, model.decoder.layers.0.self_attn.q_proj.lora_B

In [ ]:
from transformers import BartForConditionalGeneration

model = BartForConditionalGeneration.from_pretrained(model_path)  # Use the correct class
model.to(device)

BartForConditionalGeneration(
  (model): BartModel(
    (shared): BartScaledWordEmbedding(50264, 1024, padding_idx=1)
    (encoder): BartEncoder(
      (embed_tokens): BartScaledWordEmbedding(50264, 1024, padding_idx=1)
      (embed_positions): BartLearnedPositionalEmbedding(1026, 1024)
      (layers): ModuleList(
        (0-11): 12 x BartEncoderLayer(
          (self_attn): BartSdpaAttention(
            (k_proj): lora.Linear(
              (base_layer): Linear(in_features=1024, out_features=1024, bias=True)
              (lora_dropout): ModuleDict(
                (default): Dropout(p=0.05, inplace=False)
              )
              (lora_A): ModuleDict(
                (default): Linear(in_features=1024, out_features=16, bias=False)
              )
              (lora_B): ModuleDict(
                (default): Linear(in_features=16, out_features=1024, bias=False)
              )
              (lora_embedding_A): ParameterDict()
              (lora_embedding_B): ParameterDict()
   

# testing & evaluating:

testing on one doc:

In [ ]:
import torch

# Select one example from test dataset
sample = test_dataset[1]
text = sample["rohtext"]
true_summary = sample["Summarization"]

# Prepare prompt
prompt = f"<instruction>: Fasse den folgenden Text zusammen\n<document>: {text}"

# Tokenize and move to device
device = "cuda:0" if torch.cuda.is_available() else "cpu"  # Automatically select device
model.to(device)  # Move model to device

encoding = tokenizer(prompt, return_tensors="pt", truncation=True, max_length=512).to(device)

# Generate summary
with torch.inference_mode():
    outputs = model.generate(
        input_ids=encoding.input_ids,
        attention_mask=encoding.attention_mask,
        max_length=512  # Adjust as needed
    )

# Decode output
generated_summary = tokenizer.decode(outputs[0], skip_special_tokens=True)

# Print results
print("🔹 **Original Text:**", text)
print("\n✅ **Generated Summary:**", generated_summary)
print("\n🎯 **True Summary:**", true_summary)


🔹 **Original Text:** 
Guten Tag, Bautzen! Danke für die freundliche Begrüßung. Mit Blick auf den Ministerpräsidenten natürlich auch: Guten Tag, Sachsen!
Sie wissen ja, dass ich Mecklenburger bin, und wenn ich Sachsen so freundlich begrüße, dann hat das historische Gründe. Auf die komme ich gleich zu sprechen. Aber zunächst will ich mich bedanken, dass Sie alle meiner Einladung gefolgt sind. Danken möchte ich auch der Hausherrin. Hier im sorbischen National-Ensemble zu sein, freut mich. Schön, dass wir so charmante Gastgeber haben, die für die Eigenständigkeit dieser Region stehen.
"Demokratie geht auch mich etwas an!" – so haben wir diese Diskussionsrunde genannt. Es ist mir wichtig, dass sich möglichst viele Menschen unter einer Überschrift versammeln können, dass sich unterschiedliche Milieus mischen und ganz unterschiedliche Ansichten vertreten sind: der Handwerker, das Gemeindekirchenratsmitglied, die Gewerkschafterin, der Fußballtrainer, die Abiturientin, der Berufsschüler, die Fr

In [ ]:
from rouge_score import rouge_scorer
import bert_score

rouge = rouge_scorer.RougeScorer(["rouge1", "rouge2", "rougeL"], use_stemmer=True)
rouge_results = rouge.score(true_summary, generated_summary)

# Print ROUGE results
print("ROUGE Results:")
print("ROUGE-1 F1 score:", rouge_results["rouge1"].fmeasure)
print("ROUGE-2 F1 score:", rouge_results["rouge2"].fmeasure)
print("ROUGE-L F1 score:", rouge_results["rougeL"].fmeasure)

# Compute BERTScore
P, R, F1 = bert_score.score([generated_summary], [true_summary], lang="de", model_type="bert-base-multilingual-cased")

# Print BERTScore results
print("\nBERTScore Results:")
print("Precision:", P.item())  # BERT precision score
print("Recall:", R.item())     # BERT recall score
print("F1 score:", F1.item())  # BERT F1 score

ROUGE Results:
ROUGE-1 F1 score: 0.3247863247863248
ROUGE-2 F1 score: 0.10315186246418338
ROUGE-L F1 score: 0.1766381766381766


tokenizer_config.json:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/625 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/996k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.96M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/714M [00:00<?, ?B/s]


BERTScore Results:
Precision: 0.6491349339485168
Recall: 0.677936315536499
F1 score: 0.663223147392273


In [ ]:
import torch
from tqdm import tqdm
from rouge_score import rouge_scorer
import bert_score
import pandas as pd

# Initialize the ROUGE scorer
rouge = rouge_scorer.RougeScorer(["rouge1", "rouge2", "rougeL"], use_stemmer=True)

def evaluate_test_dataset(test_dataset, tokenizer, model, device="cuda:0", output_file="generated_summaries.csv"):
    # Lists to store results
    all_generated_summaries = []
    all_true_summaries = []

    rouge1_f1_scores = []
    rouge2_f1_scores = []
    rougeL_f1_scores = []

    bert_precision_scores = []
    bert_recall_scores = []
    bert_f1_scores = []

    # Iterate over the test dataset
    for sample in tqdm(test_dataset, desc="Evaluating test dataset"):
        text = sample["rohtext"]
        true_summary = sample["Summarization"]

        # Prepare the prompt
        prompt = f"<instruction>: Fasse den folgenden Text zusammen\n<document>: {text}"

        device = "cuda:0" if torch.cuda.is_available() else "cpu"  # Automatically select device
        model.to(device)

        # Tokenize and move to device
        encoding = tokenizer(prompt, return_tensors="pt", truncation=True, max_length=512).to(device)

        # Generate summary
        with torch.inference_mode():
            outputs = model.generate(
                input_ids=encoding.input_ids,
                attention_mask=encoding.attention_mask,
                max_length=400  # Adjust if needed
            )

        # Decode output
        generated_summary = tokenizer.decode(outputs[0], skip_special_tokens=True)

        # Compute ROUGE scores
        rouge_results = rouge.score(true_summary, generated_summary)

        # Store ROUGE results
        rouge1_f1_scores.append(rouge_results["rouge1"].fmeasure)
        rouge2_f1_scores.append(rouge_results["rouge2"].fmeasure)
        rougeL_f1_scores.append(rouge_results["rougeL"].fmeasure)

        # Compute BERTScore
        P, R, F1 = bert_score.score([generated_summary], [true_summary], lang="de", model_type="bert-base-multilingual-cased")

        # Store BERTScore results
        bert_precision_scores.append(P.item())
        bert_recall_scores.append(R.item())
        bert_f1_scores.append(F1.item())

        # Store the generated summary and true summary for potential future use
        all_generated_summaries.append(generated_summary)
        all_true_summaries.append(true_summary)

    # Calculate average scores
    avg_rouge1_f1 = sum(rouge1_f1_scores) / len(rouge1_f1_scores)
    avg_rouge2_f1 = sum(rouge2_f1_scores) / len(rouge2_f1_scores)
    avg_rougeL_f1 = sum(rougeL_f1_scores) / len(rougeL_f1_scores)

    avg_bert_precision = sum(bert_precision_scores) / len(bert_precision_scores)
    avg_bert_recall = sum(bert_recall_scores) / len(bert_recall_scores)
    avg_bert_f1 = sum(bert_f1_scores) / len(bert_f1_scores)

    # Save generated summaries and true summaries to a CSV file
    results_df = pd.DataFrame({
        'generated_summary': all_generated_summaries,
        'true_summary': all_true_summaries
    })
    results_df.to_csv(output_file, index=False)

    # Return aggregated metrics and the summaries DataFrame
    return {
        'rouge1_f1': avg_rouge1_f1,
        'rouge2_f1': avg_rouge2_f1,
        'rougeL_f1': avg_rougeL_f1,
        'bert_precision': avg_bert_precision,
        'bert_recall': avg_bert_recall,
        'bert_f1': avg_bert_f1,
        'summaries_df': results_df  # Return the DataFrame
    }


In [ ]:
results = evaluate_test_dataset(test_dataset, tokenizer, model, df)

# Access the CSV file with summaries
print("Summaries saved in 'updated_summaries.csv'")

# Access the updated DataFrame with generated summaries
updated_df = results["summaries_df"]
print(updated_df.head())  # Print the first few rows of the updated DataFrame


Evaluating test dataset: 100%|██████████| 750/750 [1:19:20<00:00,  6.35s/it]

Summaries saved in 'updated_summaries.csv'
                                   generated_summary  \
0  Dieser Text beschreibt Bundesaußenminister Wes...   
1  Dieser Text beschreibt eine Begrüßung des Mini...   
2  Dieser Text ist eine Rede des Bundespräsident,...   
3  Dieser Text ist eine ganz neue Aktualität auf ...   
4  Dieser Text beschreibt eine Begründung von ein...   

                                        true_summary  
0  Dies ist eine Rede des Bundesministers für Bil...  
1  Dieser Text ist ein politischer Redeschritt de...  
2  Der Text besteht aus zwei Briefen, die zwische...  
3  Die Rede ist eine Einführung in die Ausstellun...  
4  Dieser Text ist eine Rede, die sich mit dem Th...  


In [ ]:
print("Evaluation Results on Test Dataset:")
print(f"ROUGE-1 F1 score: {results['rouge1_f1']}")
print(f"ROUGE-2 F1 score: {results['rouge2_f1']}")
print(f"ROUGE-L F1 score: {results['rougeL_f1']}")
print(f"BERT Precision: {results['bert_precision']}")
print(f"BERT Recall: {results['bert_recall']}")
print(f"BERT F1 score: {results['bert_f1']}")

Evaluation Results on Test Dataset:
ROUGE-1 F1 score: 0.33350929065215457
ROUGE-2 F1 score: 0.08289087068071277
ROUGE-L F1 score: 0.17216088636878685
BERT Precision: 0.67641468334198
BERT Recall: 0.6748722964127859
BERT F1 score: 0.675298491080602


In [ ]:
len(updated_df)

750

In [ ]:
import csv

def count_rows_in_csv(file_path):
    with open(file_path, newline='', encoding='utf-8') as csvfile:
        reader = csv.reader(csvfile)
        header = next(reader)  # Skip the header row
        row_count = sum(1 for row in reader)
    return row_count

# Example usage
file_path = "generated_summaries.csv"
print(f"Number of rows: {count_rows_in_csv(file_path)}")

Number of rows: 750


In [ ]:
import csv

def count_columns_in_csv(file_path):
    with open(file_path, newline='', encoding='utf-8') as csvfile:
        reader = csv.reader(csvfile)
        # Get the first row (header) to count columns
        header = next(reader)
        return len(header)

# Example usage
file_path = "generated_summaries.csv"
print(f"Number of columns: {count_columns_in_csv(file_path)}")


Number of columns: 2


llm as judge

In [ ]:
eval = []

In [ ]:
for i in range(len(updates_df)):
    generated_summary = updated_df.loc[i, 'generated_summary']  # Access generated summary
    true_summary = updated_df.loc[i, 'true_summary']  # Access reference summary

    prompt = f"""You are an expert evaluator for summarization tasks. Compare the generated summary to the reference summary based on the following criteria:
    1. Relevance
    2. Coherence
    3. Conciseness
    4. Fluency

    Generated Summary: {generated_summary}
    Reference Summary: {true_summary}

    Provide a score for each criterion on a scale of 1 to 5 (1 = poor, 5 = excellent). Return the scores as follows
    [relevance_score, coherence_score, conciseness_score, fluency_score],
    Do not return anything else but this list with scores. Example: [0.5, 5, 4.5, 3]."""

    out = llm.invoke(prompt)
    print(f'we are at text nb: {i + 1}')  # Use i + 1 for correct text number
    eval.append(out)

we are at text nb: 1
we are at text nb: 2
we are at text nb: 3
we are at text nb: 4
we are at text nb: 5
we are at text nb: 6
we are at text nb: 7
we are at text nb: 8
we are at text nb: 9
we are at text nb: 10
we are at text nb: 11
we are at text nb: 12
we are at text nb: 13
we are at text nb: 14
we are at text nb: 15
we are at text nb: 16
we are at text nb: 17
we are at text nb: 18
we are at text nb: 19
we are at text nb: 20
we are at text nb: 21
we are at text nb: 22
we are at text nb: 23
we are at text nb: 24
we are at text nb: 25
we are at text nb: 26
we are at text nb: 27
we are at text nb: 28
we are at text nb: 29
we are at text nb: 30
we are at text nb: 31
we are at text nb: 32
we are at text nb: 33
we are at text nb: 34
we are at text nb: 35
we are at text nb: 36
we are at text nb: 37
we are at text nb: 38
we are at text nb: 39
we are at text nb: 40
we are at text nb: 41
we are at text nb: 42
we are at text nb: 43
we are at text nb: 44
we are at text nb: 45
we are at text nb: 

In [ ]:
import numpy as np
import ast

# Convert strings to lists
eval_lists = [ast.literal_eval(item) for item in eval]

# Create NumPy array
eval_array = np.array(eval_lists)

# Calculate the average for each score  (/5 to have values between 0 and 1)
avg_relevance = np.mean(eval_array[:, 0])/5
avg_coherence = np.mean(eval_array[:, 1])/5
avg_conciseness = np.mean(eval_array[:, 2])/5
avg_fluency = np.mean(eval_array[:, 3])/5

# Print the results
print(f"Average Relevance: {avg_relevance}")
print(f"Average Coherence: {avg_coherence}")
print(f"Average Conciseness: {avg_conciseness}")
print(f"Average Fluency: {avg_fluency}")

Average Relevance: 0.4549333333333333
Average Coherence: 0.5392
Average Conciseness: 0.46503999999999995
Average Fluency: 0.4343466666666667
